<a href="https://colab.research.google.com/github/bjdzliu/ai_lab/blob/main/langchain/rag_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Langchain+OpenAI+vector  
Retrival Augmented Generation  
优势：  
增加了回答专业领域内的知识  
知识库可以频繁更新  
知识库内容变更可以追踪  
数据隐私

这里包含了3个例子

firstly see a sample no-RAG

In [ ]:
!pip3 install langchain openapi chromadb   cohere tiktoken langchain_openai

In [3]:
import os
from langchain_openai import ChatOpenAI

from google.colab import userdata

apikey=userdata.get('OPENAI_API_KEY')

chat = ChatOpenAI(
    openai_api_key=apikey,
    model='gpt-3.5-turbo'
)

In [4]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
messages=[
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Knock Knock"),
    AIMessage(content="who is there"),
    HumanMessage(content="Orange")
]

In [5]:
res=chat.invoke(messages)

In [6]:
res


AIMessage(content='Orange who?')

In [7]:
messages.append(res)
res=chat(messages)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [8]:
res

AIMessage(content="Orange you glad I'm here to assist you?")

let specify some knowledge
提示工程的方式

In [9]:
baichuan2_information=[
    "百川智能在北京召开大模型发布会,正式发布Baichuan2开源大模型,昇腾AI基础软硬件平台正式支持Baichuan2大模型,并在昇思MindSpore开源社区大模型平台上线Baichuan2-7B模型开放体验。",
    "Baichuan2-7B-Base和 Baichuan2-13B-Base,均基于2.6T⾼质量多语⾔数据进⾏训练,在保留了上一代开源模型良好的生成与创作能力,流畅的多轮对话能力以及部署⻔槛较低等众多特性的基础上,两个模型在数学、代码、安全、逻辑推理、语义理解等能⼒有显著提升",
    "Baichuan2大模型是由百川智能开发的一系列开源可商用的大规模预训练语言模型"

]

In [10]:
source_knowledge='\n'.join(baichuan2_information)

In [11]:
print(source_knowledge)

百川智能在北京召开大模型发布会,正式发布Baichuan2开源大模型,昇腾AI基础软硬件平台正式支持Baichuan2大模型,并在昇思MindSpore开源社区大模型平台上线Baichuan2-7B模型开放体验。
Baichuan2-7B-Base和 Baichuan2-13B-Base,均基于2.6T⾼质量多语⾔数据进⾏训练,在保留了上一代开源模型良好的生成与创作能力,流畅的多轮对话能力以及部署⻔槛较低等众多特性的基础上,两个模型在数学、代码、安全、逻辑推理、语义理解等能⼒有显著提升
Baichuan2大模型是由百川智能开发的一系列开源可商用的大规模预训练语言模型


In [12]:
query="你知道baichuan2模型吗"

In [13]:
prompt_template=f"""基于以下内容回答问题:
内容:
{source_knowledge}
Query:
{query}
"""

In [14]:
prompt=HumanMessage(
    content=prompt_template
)
messages.append(prompt)
res=chat(messages)

In [15]:
res

AIMessage(content='是的，我知道Baichuan2模型。Baichuan2是由百川智能开发的一系列开源可商用的大规模预训练语言模型。Baichuan2-7B-Base和Baichuan2-13B-Base是其中的两个模型，它们基于2.6T高质量多语言数据进行训练。相较于上一代开源模型，Baichuan2模型在生成与创作能力、多轮对话能力、部署门槛等方面有显著提升，并在数学、代码、安全、逻辑推理、语义理解等能力上有提升。')

第三个例子：
知识库内容存放在一个pdf文件中


In [16]:
!pip3 install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 2.0 MB/s eta 0:00:00


In [18]:
from langchain.document_loaders import PyPDFLoader

In [19]:
loader=PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

In [20]:
pages=loader.load_and_split()

In [21]:
print(pages[2])

page_content='However, most open-source large language\nmodels have focused primarily on English. For\ninstance, the main data source for LLaMA\nis Common Crawl1, which comprises 67% of\nLLaMA’s pre-training data but is filtered to English\ncontent only. Other open source LLMs such as\nMPT (MosaicML, 2023) and Falcon (Penedo et al.,\n2023) are also focused on English and have limited\ncapabilities in other languages. This hinders the\ndevelopment and application of LLMs in specific\nlanguages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,\nBaichuan 2 achieves significant improvements ove

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs=text_splitter.split_documents(pages)

In [23]:
len(docs)

215

利用embeding模型对每个文本进行向量化

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embed_model=OpenAIEmbeddings(openai_api_key=apikey)
vectorstore=Chroma.from_documents(documents=docs,embedding=embed_model,collection_name="openai_embed")



In [26]:
query="How large is the baichuan2 voculary"
result=vectorstore.similarity_search(query,k=2)

In [27]:
result

[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305.pdf'}),
 Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Ni

In [ ]:
def augment_promp(query:str):
  result